In [27]:
import pandas as pd
import numpy as np
from hypex import Matcher
import warnings
import gc
warnings.simplefilter(action='ignore', category=FutureWarning)

In [40]:
data = pd.read_csv('../t_sas_green_day_data_2_202312071554.csv')

In [10]:
data.head()

epk_id  city_id  is_pilot sd_gender_cd     age  age_comp  \
0  1130266694882513050        2         1            F  76.682      76.0   
1  1127200213300749681        2         0            M  74.414      74.0   
2  1129400963242244937       14         0            F  60.553      60.0   
3  1131354787866967475        1         0            F  47.626      47.0   
4  1128617884354540037       37         0            M  31.180      31.0   

   val_up  has_active_cc  has_active_deposit_curr_acc  \
0    0.23              0                            1   
1    0.00              0                            1   
2    2.33              0                            1   
3    0.26              0                            0   
4    0.14              0                            0   

   has_active_credit_or_mortgage  dep_acct_tot_davg_yr_rub_amt  avg_income  \
0                              0                     508566.55    41306.88   
1                              0                      20837.34    22001.10   
2                              0                     666110.08        0.00   
3                              0                      33176.66        0.00   
4                              0                      12512.52        0.00   

   cnt_visits_city  crd_otf_cash_rub_amt  spend  lne_tot_debt_due_rub_amt  \
0               10                   0.0     35                       0.0   
1               14                   0.0      0                       0.0   
2               20              171700.0     44                       0.0   
3                8                   0.0      0                       0.0   
4               41              168100.0    144                       0.0   

   cnt_mnth_sbol_login  pnl_oi_total_amt  npv  is_female  
0                    1           1186.35    0          1  
1                    0             53.06    0          0  
2                    1           2422.99    0          1  
3                    1            246.87    0          1  
4                    1           1661.69    0          0

In [41]:
data.rename(columns = {'crd_otf_cash_rub_amt': 'cash_out_amt'}, inplace = True)

In [50]:
def data_prep_within_city(city_id_lst, data_prepared_raw):
    data_prepared_pd_all = pd.DataFrame()
    for city_id in city_id_lst:
        data_prepared_pd = data_prepared_raw[data_prepared_raw['city_id'] == city_id].reset_index()  
        avg_age = data_prepared_pd['age'].mean()
        data_prepared_pd['age'] = data_prepared_pd['age'].fillna(avg_age)
        age_95 = data_prepared_pd[data_prepared_pd['age'] > 0]['age'].quantile(0.95)
        val_up_95 = data_prepared_pd[data_prepared_pd['val_up'] > 0]['val_up'].quantile(0.95) 
        cash_out_95 = data_prepared_pd[data_prepared_pd['cash_out_amt'] > 0]['cash_out_amt'].quantile(0.95) 
        spend_95 = data_prepared_pd[data_prepared_pd['spend'] > 0]['spend'].quantile(0.95)
        avg_income_95 = data_prepared_pd[data_prepared_pd['avg_income'] > 0]['avg_income'].quantile(0.95) 
        cnt_visits_city_95 = data_prepared_pd[data_prepared_pd['cnt_visits_city'] > 0]['cnt_visits_city'].quantile(0.95) 
        lne_tot_debt_due_95 = data_prepared_pd[data_prepared_pd['lne_tot_debt_due_rub_amt'] > 0]['lne_tot_debt_due_rub_amt'].quantile(0.95) 
        pl_neg_95 = data_prepared_pd[data_prepared_pd['pnl_oi_total_amt'] < 0]['pnl_oi_total_amt'].quantile(0.05)
        pl_pos_95 = data_prepared_pd[data_prepared_pd['pnl_oi_total_amt'] > 0]['pnl_oi_total_amt'].quantile(0.95)
        dep_bal_neg_95 = data_prepared_pd[data_prepared_pd['dep_acct_tot_davg_yr_rub_amt'] < 0]['dep_acct_tot_davg_yr_rub_amt'].quantile(0.05)
        dep_bal_pos_95 = data_prepared_pd[data_prepared_pd['dep_acct_tot_davg_yr_rub_amt'] > 0]['dep_acct_tot_davg_yr_rub_amt'].quantile(0.95)   
        data_prepared_pd['age'] = [x if x < age_95 else age_95 for x in data_prepared_pd['age']]
        data_prepared_pd['val_up'] = [x if x < val_up_95 else val_up_95 for x in data_prepared_pd['val_up']]
        data_prepared_pd['cash_out_amt'] = [x if x < cash_out_95 else cash_out_95 for x in data_prepared_pd['cash_out_amt']]
        data_prepared_pd['spend'] = [x if x < spend_95 else spend_95 for x in data_prepared_pd['spend']]
        data_prepared_pd['avg_income'] = [x if x < avg_income_95 else avg_income_95 for x in data_prepared_pd['avg_income']]
        data_prepared_pd['cnt_visits_city'] = [x if x < cnt_visits_city_95 else cnt_visits_city_95 for x in data_prepared_pd['cnt_visits_city']]
        data_prepared_pd['lne_tot_debt_due_rub_amt'] = [x if x < lne_tot_debt_due_95 else lne_tot_debt_due_95 for x in data_prepared_pd['lne_tot_debt_due_rub_amt']]
        data_prepared_pd['pnl_oi_total_amt'] = [pl_neg_95 if x < pl_neg_95 else pl_pos_95 if x > pl_pos_95 else x for x in data_prepared_pd['pnl_oi_total_amt']]
        data_prepared_pd['dep_acct_tot_davg_yr_rub_amt'] = [dep_bal_neg_95 if x < dep_bal_neg_95 else dep_bal_pos_95 if x > dep_bal_pos_95 else x 
                                                       for x in data_prepared_pd['dep_acct_tot_davg_yr_rub_amt']]       
        if data_prepared_pd.shape[0] > 0:
            data_prepared_pd_all = data_prepared_pd_all.append(data_prepared_pd)     
        del data_prepared_pd
        gc.collect()
    return data_prepared_pd_all

In [51]:
cities_list = list(data.city_id.unique())

In [52]:
data_prepared_pd_all = data_prep_within_city(cities_list, data)

In [53]:
data_prepared_pd_all.columns

Index(['index', 'epk_id', 'city_id', 'is_pilot', 'sd_gender_cd', 'age',
       'age_comp', 'val_up', 'has_active_cc', 'has_active_deposit_curr_acc',
       'has_active_credit_or_mortgage', 'dep_acct_tot_davg_yr_rub_amt',
       'avg_income', 'cnt_visits_city', 'cash_out_amt', 'spend',
       'lne_tot_debt_due_rub_amt', 'cnt_mnth_sbol_login', 'pnl_oi_total_amt',
       'npv', 'is_female'],
      dtype='object')

In [56]:
group_col = "city_id"
outcome = 'npv'
treatment = 'is_pilot'
info_col = ['epk_id']

In [57]:
model = Matcher(input_data = data_prepared_pd_all, outcome = outcome, treatment = treatment, info_col = info_col, group_col = group_col)
results, quality_results, df_matched = model.estimate(features = ['is_female', 'age', 'val_up', 'has_active_cc', 'has_active_deposit_curr_acc',
       'has_active_credit_or_mortgage', 'dep_acct_tot_davg_yr_rub_amt', 'avg_income', 'cnt_visits_city', 'cash_out_amt', 'spend',
       'lne_tot_debt_due_rub_amt', 'cnt_mnth_sbol_login', 'pnl_oi_total_amt' ])

[07.12.2023 16:16:27 | hypex | INFO]: Number of NaN values filled with zeros: 61


  0%|          | 0/30 [00:00<?, ?it/s]

KeyError: 'is_pilot'